<a href="https://colab.research.google.com/github/PashaIanko/Kaggle.Titanic/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [2]:
# This is a first commit from google colab